In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns

In [ ]:
df = pd.read_csv('../dataset/clean/sample_sorted.csv', header=0)

In [67]:
c0 = [3, 5, 6] # Looking at just cluster 0

In [68]:
# Only select pid's in that cluster
df_sub = df.where(df.pid.isin(c0))
df_sub = df_sub[~df_sub.pid.isna()]
df_sub['pid'] = df_sub['pid'].astype(int)
df_sub['date'] = pd.to_datetime(df_sub['date'])

In [71]:
# Match up all participants along columns by date. Make new columns for each participant
combined_df = None
for pid, pid_df in df_sub.groupby('pid'):
    pid_df = pid_df.rename(columns={x: x+'_'+str(pid) for x in pid_df.columns if x != 'date'})
    pid_df = pid_df.where(~pid_df['date'].isna()).drop_duplicates('date')
    if combined_df is None:
        combined_df = pid_df
    else:
        combined_df['date'] = pd.to_datetime(combined_df['date'])
        pid_df['date'] = pd.to_datetime(pid_df['date'])
        combined_df = combined_df.merge(pid_df, on='date', how='inner')
        

In [72]:
# Calculate correlations and unstack them so we can look at individual values and pairs
corrs = combined_df.corr()
s = corrs.unstack()
so = s.sort_values(ascending=False)

In [73]:
# Print rules only where the features match
exists = set()
for pair, corr in so.items():
    # Repeated pair
    if frozenset(pair) in exists: continue
    l, r = pair
    # PID matches, not interesting
    if l.split('_')[-1] == r.split('_')[-1]: continue
    # Different f eatures
    if ''.join(l.split('_')[:-1]) != ''.join(r.split('_')[:-1]): continue
    
    exists.add(frozenset(pair))
    print(pair, corr)

('survey_stress_5', 'survey_stress_6') 0.5820518408778235
('total_calories_6', 'total_calories_3') 0.3878723436235288
('vaminutes_3', 'vaminutes_6') 0.37064675220100674
('hr_std_6', 'hr_std_5') 0.367792035590586
('total_calories_5', 'total_calories_3') 0.34881634407474776
('hr_max_5', 'hr_max_6') 0.34673996889186504
('rem_sleep_count_6', 'rem_sleep_count_3') 0.341733978472694
('total_steps_6', 'total_steps_3') 0.34039150292375653
('hr_min_3', 'hr_min_5') 0.33237678203383497
('hr_max_3', 'hr_max_6') 0.31558732863227107
('total_distance_3', 'total_distance_6') 0.3092762983773614
('total_steps_5', 'total_steps_3') 0.2797266112536312
('laminutes_6', 'laminutes_3') 0.2680435158097036
('wake_sleep_count_3', 'wake_sleep_count_6') 0.2608741405750799
('total_distance_5', 'total_distance_3') 0.2607907759589671
('resthr_5', 'resthr_6') 0.25316887842039487
('hr_mean_5', 'hr_mean_6') 0.2510273666394891
('hr_std_3', 'hr_std_6') 0.24652014296223973
('hr_median_6', 'hr_median_5') 0.24340736647296743
(